In [1]:
import torch

import matplotlib.pyplot as plt

from datasets.openml_datasets import OpenML_Dataset
from datasets.torch_datasets import Torch_Dataset
from utils.weakener import Weakener
from models.model import MLP
from models.general_model import MLP
from utils.losses import PartialLoss,LBLoss,EMLoss,OSLCELoss,OSLBrierLoss,CELoss
from utils.trainig_testing import train_model,evaluate_model,train_and_evaluate

In [10]:
#[mnist_random.pt]
mlp = torch.load('results/mnist_random')

ModuleNotFoundError: No module named 'models.mlp'

In [2]:
Data = Torch_Dataset('mnist')
train_x, train_y, test_x, test_y = Data.get_data()
# WL stores processes relative to the Weakening process
Weak = Weakener(Data.num_classes)
Weak.generate_M(model_class='pll')

In [3]:
z, w = Weak.generate_weak(train_y)

In [4]:
Data.include_weak(Weak.w)

In [5]:
trainloader, testloader = Data.get_dataloader()

In [6]:
mlp = MLP(Data.num_features,[Data.num_features,Data.num_features,Data.num_features],Data.num_classes,dropout_p=0.3)
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.001)
loss_fn = OSLCELoss()

#mlp,results = train_and_evaluate(mlp,trainloader,testloader,optimizer,loss_fn,num_epochs=10)

In [11]:
import pickle
with open('results/mnist_random.pkl', 'rb') as file:
    models = pickle.load(file)


#mlp = model_data['model']
#results = model_data['results']

ModuleNotFoundError: No module named 'models.mlp'

In [3]:
model_data

NameError: name 'model_data' is not defined

In [7]:
MLP()

TypeError: __init__() missing 3 required positional arguments: 'input_size', 'hidden_sizes', and 'output_size'